# Changepoints

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (16:39:32) Alles bereit 


In [2]:
def gen_changepoints(period_len=None,incl_words=True,
                     lim=None,num_proc=1,force=False,force_data=False,
                     only_peak_scores=True,only_first_peak=False,
                     min_word_len=4,min_num_rows=15,p_peak=1,
                     min_peaks=1,consecutive_peaks=True,
                     **signif_nov_sc_opts):
    
    defaults=dict(
        only_peak_scores=only_peak_scores,
        only_first_peak=only_first_peak,
        min_foote_size=FOOTE_W,
        max_foote_size=FOOTE_W,
        p_peak=p_peak,
        min_peaks=min_peaks,
        force=force_data,
        consecutive_peaks=consecutive_peaks,
        min_word_len=min_word_len,
        min_num_rows=min_num_rows
    )
    opts={**defaults, **signif_nov_sc_opts}
    key=f'{sorted(opts.items())}/incl_words={incl_words}__lim={lim}'
    if not force:
        with get_db('changepoint') as db:
            if key in db: return db[key]
    
#     pprint(opts)
    odf=get_signif_novelty_scores(**opts).sort_values('novdist_rank')
#     return odf.set_index('word')
    
    if period_len:
        odf['period'] = (odf['period']//period_len) * period_len
    #odf=odf.groupby(['word','period']).mean().reset_index()
    #odf=odf.sort_values('novdist_rank').drop_duplicates('word')
    #odf=odf.sort_values('period').drop_duplicates('word')
    if not 'pos' in set(odf.columns): odf=odf.merge(get_word2pos_df(),on='word')
    #odf['pos']=[x[0] for x in odf.pos]
    odf=odf.set_index('word')
    odf['novdist_rank_word']=odf.novdist_rank.rank()
    odf=odf.sort_values('novdist_rank_word')
    if incl_words:
        #worddf=pd.concat(nbr_changepoint(w,cp) for w,cp in tqdm(zip(odf.index, odf.period), total=len(odf)))
        objs=list(zip(odf.index, odf.period))[:lim]
        worddf=pd.concat(pmap_iter(_get_changepoints_, objs, num_proc=num_proc))
        odf=odf.join(worddf,how='left',rsuffix='_nbr')
        
    with get_db('changepoint','w') as db: db[key]=odf
    return odf

def _get_changepoints_(obj): return nbr_changepoint(*obj)

In [3]:
# o1=gen_changepoints(period_len=5,force=True,only_first_peak=True,only_peak_scores=False)
# o1
# # o2=get_changepoints(period_len=5,force=True,only_first_peak=False,only_peak_scores=True)
# # # o1==o2

In [4]:
# dfnov=get_all_novelty_scores()
# dfnov

In [5]:
def get_changepoint_dist(w,dfch=None,dfdist=None,**args):
    o=pd.Series(dict(word=w))
    if dfch is None: dfch=get_changepoints(p_peak=1,force=True,**args)
    if not w in set(dfch.index): return o
    row=dfch.loc[w]
    chp=row.period
    if dfdist is None:
        dfdist=distmat(w)
    if not len(dfdist): return o
    crossdist = dfdist.loc[
        [ix for ix in dfdist.index if ix<str(chp)]
    ][
        [cx for cx in dfdist.columns if cx>str(chp)]
    ]
    o['chp']=chp
    o['chpdist']=crossdist.mean().mean()
#     for k,v in row.items(): o[k]=v
    return o


In [6]:
# get_changepoint_dist('virtue')

In [7]:
def get_changepoint_dists(words,num_proc=1,**opts):
    opts['dfch']=get_changepoints(p_peak=1,force=True,**opts)
    return pd.DataFrame(
        pmap_iter(
            get_changepoint_dist,
            to_words(words),
            kwargs=opts,
            num_proc=num_proc
        )    
    )

In [8]:
def get_changepoint_dists_all(
        ymin=YMIN_DISTMAT,
        ymax=YMAX_DISTMAT,
        ybin=YBIN_DISTMAT,
        k=K,
        force=False,
        lim=None,
        **opts
        ):
    suf=f'{ymin}-{ymax}_{ybin}/k={k}'
    key=f'changepoint_dist/{suf}'
    odf=None
    with get_db('changepoint') as db:
        if not force and key in db:
            odf=db[key]
    
    if odf is None:
        dfch=gen_changepoints(p_peak=1,min_periods=1,force=True,**opts)
        o=[]
        with get_db('distmat','r') as db:
            dkw=[k for k in db.keys() if k.endswith(suf)]
            for i,qstr in enumerate(tqdm(dkw,desc='Gathering distances across changepoints')):
                w=qstr.split('/')[0]
                odx = get_changepoint_dist(w,dfch=dfch,dfdist=db[qstr])
                if len(odx)>1: o.append(odx)
                if lim and i>=lim: break
        odf=pd.DataFrame(o)
        
        odf['chpdist_rank']=odf.chpdist.rank(ascending=False)
        odf['chpdist_perc']=odf.chpdist.rank(ascending=True) / len(odf) * 100
        odf=odf.sort_values('chpdist_rank')
        
        with get_db('changepoint','w') as db:
            db[key]=odf
    
    
    return odf.set_index('word')

In [9]:
# # odf_dists=get_changepoint_dists('culture,virtue,value')
# odf_dists=get_changepoint_dists(get_valid_words(),num_proc=4)
# odf_dists

In [26]:
# # odf_dists = get_changepoint_dists_all(force=True)
# odf_dists = get_changepoint_dists_all(force=False)
# odf_dists

In [27]:
def get_changepoints(**opts):
    odf1=gen_changepoints(**opts)
    odf2=get_changepoint_dists_all(**opts)
    odf=odf2.join(odf1)
    return odf

In [30]:
get_changepoints(num_proc=6)

,chp,chpdist,chpdist_rank,chpdist_perc,period,foote_novelty,foote_size,p_peak,p_trough,foote_novelty_z,...,selfdist_rank,novdist_perc_avg,novdist_perc,novdist_rank,word_num_rows,word_num_peaks,novdist_rank_word,changepoint,neighb1,neighb2
word,,,,,,,,,,,,,,,,,,,,,
satin,1790,0.371287,1.0,100.000000,1790,13.251035,6,0.00,1.0,9.872929,...,2,99.986327,99.999602,3.0,33,33,1.0,1790,"**-laced** (3), **-sapphire** (2), **-cushion** (2), **-chef** (2), **-boiler** (2), **-jester** (2), **-attired** (2), **-crane** (2), **-glittered** (2), **-sob** (2), **-nom** (2), **-jeweller** (2), **-inlaid** (2), **-wielded** (1), **-doc** (1), **-wrists** (1), **-eels** (1), **-toilet** (1), **-tau** (1), **-baroness** (1), **-larder** (1), **-varnished** (1), **-gala** (1), **-physiology** (1), **-gel** (1), **-thrusting** (1), **-delicacies** (1), **-peel** (1), **-holders** (1), **-liner** (1), **-halo** (1), **-shopkeeper** (1), **-salmon** (1), **-slit** (1), **-eucharist** (1), **-everyday** (1), **-mirrors** (1), **-harriers** (1), **-eta** (1), **-lobby** (1), **-pith** (1), **-hairy** (1), **-mantles** (1), **-academic** (1), **-rug** (1), **-farm** (1), **-benedictine** (1), **-wigs** (1), **-silvery** (1), **-shroud** (1)","velvet (42), trimmed (42), pink (39), embroidered (39), **+lace** (31), **+silk** (31), ribbon (29), muslin (27), striped (27), **+sleeves** (26), **+skirt** (24), **+bodice** (22), **+tulle** (19), **+chiffon** (18), brocade (18), **+trimming** (16), embroidery (15), gown (15), ribbons (15), crape (15), scarlet (14), **+crepe** (13), **+corsage** (13), **+feathers** (12), **+draped** (12), gauze (13), **+ornamented** (11), **+frock** (11), **+blouse** (11), **+lined** (11), **+collar** (11), jacket (11), **+crimson** (10), **+edged** (10), fringe (11), **+pleated** (9), **+scarf** (9), **+pearls** (9), **+yellow** (8), **+cambric** (8), **+mauve** (8), **+gloves** (8), petticoat (8), tunic (8), white (8), **+ivory** (7), **+pearl** (7), blond (7), bordered (7), blue (7)"
factor,1875,0.369789,2.0,99.993147,1875,12.731170,6,0.00,1.0,9.426598,...,5,99.972291,99.998208,10.0,36,36,5.0,1875,"**-grocer** (27), **-tanner** (20), **-stationer** (20), **-dealer** (19), **-broker** (19), **-innkeeper** (19), **-merchant** (18), **-baker** (17), **-shopkeeper** (15), **-wilts** (14), **-builder** (14), **-jeweller** (14), **-bankrupts** (13), **-weaver** (13), **-manufacturer** (12), **-butcher** (12), **-carpenter** (12), **-tailor** (10), **-derby** (10), **-bucks** (10), **-walker** (9), **-grocers** (9), **-banker** (9), **-cardigan** (7), **-seller** (6), **-shire** (5), **-brewers** (5), **-drake** (5), **-carver** (5), **-partners** (5), **-maker** (5), **-shoemaker** (5), **-clerk** (4), **-chinaman** (4), **-warren** (4), **-sheriff** (4), **-downing** (4), **-carrier** (4), **-calico** (4), **-solicitors** (4), **-stoke** (4), **-milliner** (4), **-chant** (4), **-sandwich** (4), **-moray** (4), **-publican** (4), **-linen** (4), **-tenant** (4), **-marshall** (4), **-goldsmith** (3)","element (19), **+development** (15), **+economic** (14), **+principle** (14), **+organisation** (12), **+unity** (11), **+industry** (11), **+fundamental** (11), **+activity** (10), function (10), **+organization** (9), **+vital** (9), **+outcome** (9), **+expansion** (8), product (8), **+organism** (7), **+problem** (7), **+constitution** (7), **+essential** (6), **+industrial** (6), **+basis** (6), **+necessity** (6), **+economy** (5), **+conception** (5), **+inherent** (5), **+community** (5), **+policy** (4), **+possibility** (4), **+universal** (4), **+stability** (4), **+nation** (4), **+aspect** (4), **+effective** (3), **+method** (3), **+process** (3), **+formula** (3), **+involved** (3), **+morality** (3), **+democracy** (3), **+achievement** (3), **+evolution** (3), **+enterprise** (3), **+structure** (3), **+ultimate** (3), **+equilibrium** (3), **+solution** (3), **+phenomenon** (3), **+device** (3), **+doctrine** (3), **

In [13]:
# dfnov.query('word=="harmony" & period<1900')

In [14]:
# nbr_changepoint('performance',1810)

In [15]:
def show_changepoints(df=None,eg_lim=100,period_len=20,no_fs=False,
                      min_novdist_perc=None,min_dist_perc=None,min_nov_perc=None,
                      **opts):
    invalid=get_invalid_words()
    abswords=set(get_words_ever_abs(cutoff=1.5))
    odf=get_changepoints(period_len=period_len,**opts).reset_index() if df is None else df
    if no_fs:
        for ltr in ['f','s']:
            odf=odf[~odf.word.str.contains(ltr)]
#     odf['pos']=odf.pos.apply(lambda x: x[0])
    odf=odf[~odf.word.isin(invalid)]
    if min_novdist_perc: odf=odf[odf.novdist_perc>=min_novdist_perc]
    if min_dist_perc: odf=odf[odf.selfdist_perc>=min_dist_perc]
    if min_nov_perc: odf=odf[odf.foote_novelty_perc>=min_nov_perc]
    
    o=[]
    for period,g in tqdm(odf.groupby('period')):
        periodstr=f'{period}-{period+period_len}'
        od=dict(period=periodstr)
        
        for pos,gpos in g.groupby('pos'):
            pwords=gpos.sort_values('novdist_rank').word.iloc[:eg_lim]
            pabswords=[w for w in pwords if w in abswords]
            pconcwords=[w for w in pwords if w not in abswords]
            wordstrs=[]
            if pabswords: wordstrs.append(f"""<i>Abstract</i>: {', '.join(pabswords)}""")
            if pconcwords: wordstrs.append(f"""<i>Other</i>: {', '.join(pconcwords)}""")
            wordstr='<br/><br/>'.join(wordstrs)
            od[pos]=wordstr
        o.append(od)
    odf=pd.DataFrame(o).set_index('period').sort_index().fillna('')
    odf=odf[['nn1','nn2','j','v']]
    odf.columns=[POSD.get(x) for x in odf.columns]
#     odf=odf[['n','j','v']]
#     odf.columns=['Nouns','Adjectives','Verbs']
    return odf
        

In [16]:
# get_words_ever_abs??

In [18]:
# odf=show_changepoints(eg_lim=200,period_len=20,min_peaks=1,p_peak=.01,
#                      min_dist_perc=75,force=True,force_data=True)
# # odf
# printm(odf)

In [112]:
# odf.style.set_properties(**{'width':'100px'})
# # odf.style

In [19]:
# df_to_png(odf, os.path.join(PATH_FIGS2,'fig.pivotpoints3.png'))

In [20]:
# get_changepoints().loc['philosophical']

In [21]:
# dfch=get_changepoints()
# dfch['changepoint']=dfch.period
# dfch

In [22]:
def periodise2(y):
    if y>=1790 and y<1820: return 'Sattelzeit I'
    if y>=1890 and y<1920: return 'Sattelzeit II'
    return ''

In [25]:
def nbr_changepoint(w,changepoint=None,ntop=50,k=25,ybins=[5,20],**opts):
    odx=dict(word=word,changepoint=changepoint)
    
    if not changepoint:
        try:
            changepoint=get_changepoints(incl_words=False).loc[w].period
        except KeyError:
            return pd.DataFrame([odx]).set_index('word')
            
    key=lltk.hashstr(str([w,changepoint,ntop,k,ybins,sorted(opts.items())]))[:11]
    with get_db('changepoint') as db:
        if key in db: return db[key]
            
            
    dfnbr=pd.concat(
        nbr(w,ybin=ybin,k=k,**opts).reset_index().query(f'rank<={k}')
        for ybin in ybins
    )
    dfnbr['count']=1
    dfnbr1 = dfnbr.query(f'period<"{changepoint}"')
    dfnbr2 = dfnbr.query(f'period>="{changepoint}"')
    dfnbr1s = dfnbr1.groupby('neighbor').sum().sort_values(
        ['count','rank','dist'],ascending=[0,1,1]
    )#.iloc[:k]
    dfnbr2s = dfnbr2.groupby('neighbor').sum().sort_values(
        ['count','rank','dist'],ascending=[0,1,1]
    )#.iloc[:k]
    l1=dfnbr1s.index
    l2=dfnbr2s.index
    raw1=dfnbr1s['count']
    raw2=dfnbr2s['count']
    c1=((raw1/len(dfnbr1s))*1000)#.apply(int)
    c2=((raw2/len(dfnbr2s))*1000)#.apply(int)
    set1=set(l1)#[:k])
    set2=set(l2)#[:k])
    w2c1=Counter(dict(zip(l1,c1)))
    w2c2=Counter(dict(zip(l2,c2)))
    w2raw1=Counter(dict(zip(l1,raw1)))
    w2raw2=Counter(dict(zip(l2,raw2)))
    
    ol1=[
        (w,c)
        for w,c in sorted(
            zip(l1, c1),
            key = lambda xy: w2c2[xy[0]] - xy[1]
        ) if c>=w2c2[w]
    ]
    ol2=[
        (w,c)
        for w,c in sorted(
            zip(l2, c2),
            key = lambda xy: w2c1[xy[0]] - xy[1]
        ) if c>=w2c1[w]
    ]
    
    
    ostr1=', '.join(
        #f'{"**-" if w not in set2 else ""}{w}{"**" if w not in set2 else ""} ({(w2c2[w]-c):+.0f})'
        f'{"**-" if w not in set2 else ""}{w}{"**" if w not in set2 else ""} ({w2raw1[w]})'
        for w,c in ol1[:ntop]
    )
    
    ostr2=', '.join(
        #f'{"**+" if w not in set1 else ""}{w}{"**" if w not in set1 else ""} ({(c-w2c1[w]):+.0f})'
        f'{"**+" if w not in set1 else ""}{w}{"**" if w not in set1 else ""} ({w2raw2[w]})'
        for w,c in ol2[:ntop]
    )
    odx=dict(
        word=w,
        changepoint=changepoint,
        neighb1=ostr1,
        neighb2=ostr2,
    )
    odf=pd.DataFrame([odx]).set_index('word')
    with get_db('changepoint','w') as db: db[key]=odf
    return odf

printm(nbr_changepoint('complexions',k=25,ntop=50))

| word        |   changepoint | neighb1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | neighb2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|:------------|--------------:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| complexions |          1910 | **-plumage** (14), **-countenances** (10), **-auburn** (9), **-tawny** (8), **-beards** (8), **-dresses** (8), tints (12), **-fringed** (7), **-veils** (7), **-variegated** (7), **-bracelets** (7), tresses (11), **-colour** (6), **-faces** (6), **-plumes** (6), **-swarthy** (6), **-mantles** (6), **-berries** (6), **-dyes** (6), **-cloaks** (6), **-draperies** (6), **-gowns** (6), **-attitudes** (6), **-ostrich** (6), **-tempers** (5), **-shapes** (5), **-perfumes** (5), **-fringe** (5), **-verdure** (5), **-caps** (5), **-butterflies** (5), **-hue** (4), **-features** (4), **-pearly** (4), **-eyebrows** (4), **-clusters** (4), **-bouquets** (4), **-pink** (4), **-shaped** (4), **-wreaths** (4), **-colours** (4), **-brocade** (4), **-bonnets** (4), hues (8), **-earrings** (3), **-sapphire** (3), **-trimmings** (3), **-tastes** (3), **-lilac** (3), **-frocks** (3) | complexion (11), skin (8), **+blemishes** (6), **+greasy** (6), tissues (6), wrinkles (6), roughness (6), **+powders** (5), skins (6), nourishing (5), cleanse (5), fragrant (5), softness (5), **+supple** (4), **+oily** (4), bloom (4), lustrous (5), **+soothing** (3), **+removes** (3), **+dryness** (3), **+cleansing** (3), **+tones** (3), **+lather** (3), **+lustre** (3), **+stains** (3), perfumed (3), tinted (3), scalp (3), lotion (3), gums (3), **+restores** (2), **+dewy** (2), **+glowing** (2), **+loveliness** (2), **+freshness** (2), **+nourish** (2), **+pores** (2), **+ingredients** (2), **+pale** (2), **+delicate** (2), **+colouring** (2), **+soft** (2), **+powder** (2), **+oils** (2), **+youthful** (2), whiteness (4), sheen (2), lashes (2), fragrance (2), antiseptic (2) |

In [29]:
# wdist??

In [30]:
# printm(nbr_changepoint('culture'))

In [54]:
# odf=get_changepoints(num_proc=4,lim=None,force=True,incl_words=True)
# # odf=get_changepoints(incl_words=True)
# odf

In [55]:
# show_changepoints()

In [56]:
# printm(nbr_changepoint('philosophy'))

# New anno